In [11]:
from ollama import chat
from pydantic import BaseModel, Field
from typing import List, Literal, Union, Dict
import json

In [28]:
AVAILABLE_MODELS = {
    'llama3.3': 'llama3.3',
    'llama3.3-q8': 'llama3.3:70b-instruct-q8_0',
    'gemma3_fp16': 'gemma3:27b-it-fp16',
    'gemma3_q8': 'gemma3:27b-it-q8_0',
}

DEFAULT_MODEL = 'gemma3_q8'

model = AVAILABLE_MODELS[DEFAULT_MODEL]

In [25]:
HADM_ID = 131407.0

In [30]:
# admission medication extraction, use structured output
class AdmissionMeds(BaseModel):
    medications: List[str] = Field(description="List of medications with dosage forms, e.g. 'Celexa 20mg capsule'")

admission_meds = json.load(open(f'results/notes/admission_meds_{HADM_ID}.json'))

meds_prompt = f"""
Extract all medications from the below json file, including the dosage form and dosage strength. e.g. 'Celexa 20mg capsule'
Medications:
{admission_meds}
"""

response = chat(
  messages=[
    {
      'role': 'user',
      'content': meds_prompt,
    }
  ],
  model=model,
  format=AdmissionMeds.model_json_schema(),
)

admission_med_list = AdmissionMeds.model_validate_json(response.message.content)
print(admission_med_list)

medications=['Cellcept 0.50 gm QAM', 'Cellcept 0.75 gm QPM', 'Prednisone 7.5 mg daily', 'Cyclosporine 70 mg', 'Nifedipine ER 30 mg daily']


In [29]:
# extract scans from restuls
class Scans(BaseModel):
    scans: Dict[str, str] = Field(description="Dictionary of scans with date and resutls, e.g. {'2124-01-01': 'CT CAP: 1. Extensive retroperitoneal...'}")

scan_notes = json.load(open(f'results/notes/scans_{HADM_ID}.json'))

scan_prompt = f"""
Extract all medical imaging scan results and date from the below json file, e.g. {{'2134-01-01': 'CT CAP: 1. Extensive retroperitoneal...'}}
Scans:
{scan_notes}
"""

response = chat(
  messages=[
    {
      'role': 'user',
      'content': scan_prompt,
    }
  ],
  model=model,
  format=Scans.model_json_schema(),
)

scan_dict = Scans.model_validate_json(response.message.content)
print(scan_dict)

scans={'2141-04-15': 'CT CAP: 1. Extensive retroperitoneal and mesenteric lymphadenopathy in addition to enlarged lymph nodes posterior to the descending thoracic aorta. In the setting of recent renal transplant, these findings are worrisome for post-transplant lymphoproliferative disorder or a lymphoma. 2. Mild ascites. 3. Right shoulder lipoma. 4. Small bilateral pleural effusions. CT Neck - Unremarkable', '2141-04-17': "ECHO - The left atrium is dilated. The right atrium is moderately dilated. Left ventricular wall thickness, cavity size and regional/global systolic function are normal (LVEF >55%). Due to suboptimal technical quality, a focal wall motion abnormality cannot be fully excluded. The right ventricular cavity is mildly dilated with normal free wall contractility. The aortic valve leaflets (3) are mildly thickened but aortic stenosis is not present. No masses or vegetations are seen on the aortic valve, but cannot be fully excluded due to suboptimal image quality. The mitr

In [33]:
HADM_ID = 29463

# modify events to exclude any mentioning of medications
class ModifiedEvent(BaseModel):
    modified_text: str = Field(description="Event text with medication mentions replaced")

events = json.load(open(f'results/notes/events_{HADM_ID}.json'))

processed_events = []

for event in events:
    event_text = event.get('events', '')
    
    single_event_prompt = f"""
    Modify the following hospital event text by replacing any mentions of specific medications with the generic word 'medication'.
    Do not remove or alter any other information such as dates, vitals, or non-medication related events.
    
    Event Text:
    {event_text}
    """
    
    response = chat(
        messages=[
            {
                'role': 'user',
                'content': single_event_prompt,
            }
        ],
        model=model,
        format=ModifiedEvent.model_json_schema(),
    )
    
    result = ModifiedEvent.model_validate_json(response.message.content)
    
    # Create a copy of the original event and update only the events field
    modified_event = event.copy()
    modified_event['events'] = result.modified_text
    processed_events.append(modified_event)

# Print the first processed event as an example
print("Sample processed event:")
print(processed_events[1])

Sample processed event:
{'admit_time': '2153-04-24 15:55:00', 'discharge_time': '2153-04-30', 'event_time': '2153-04-30 08:00:00', 'vitals': 'Tmax: 37,C (98.6,   Tcurrent: 36.7,C (98,   HR: 77 (76 - 85) bpm,   BP: 134/61(78) {111/57(73) - 144/71(84)} mmHg,   ', 'events': '24 Hour Events:, CALLED OUT,  -called out,  -Surgery recommended to discontinue medication and re-start home, medication,'}
